<a href="https://colab.research.google.com/github/abinayaa-148/Bio---Silicon-Labs/blob/main/Global_alignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

# Scoring setup
Match_score = 1
Mismatch_penalty = -1
Gap_penalty = -2

def needleman_wunsch(seq1, seq2):
    # Initialize scoring matrix
    m = len(seq1)
    n = len(seq2)
    score_matrix = np.zeros((m+1, n+1), dtype=int)

    # Fill the first row and column with gap penalties
    for i in range(m + 1):
        score_matrix[i][0] = Gap_penalty * i
    for j in range(n + 1):
        score_matrix[0][j] = Gap_penalty * j

    # Fill the scoring matrix
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            match = score_matrix[i - 1][j - 1] + (Match_score if seq1[i - 1] == seq2[j - 1] else Mismatch_penalty)
            delete = score_matrix[i - 1][j] + Gap_penalty
            insert = score_matrix[i][j - 1] + Gap_penalty
            score_matrix[i][j] = max(match, delete, insert)

    # Convert to DataFrame for better visualization
    df_score = pd.DataFrame(score_matrix, index=["-"] + list(seq1), columns=["-"] + list(seq2))

    # Traceback to get alignment
    align1, align2 = "", ""
    i, j = m, n

    while i > 0 and j > 0:
        current_score = score_matrix[i][j]
        diagonal = score_matrix[i - 1][j - 1]
        up = score_matrix[i - 1][j]
        left = score_matrix[i][j - 1]

        if current_score == diagonal + (Match_score if seq1[i - 1] == seq2[j - 1] else Mismatch_penalty):
            align1 = seq1[i - 1] + align1
            align2 = seq2[j - 1] + align2
            i -= 1
            j -= 1
        elif current_score == up + Gap_penalty:
            align1 = seq1[i - 1] + align1
            align2 = "-" + align2
            i -= 1
        else:
            align1 = "-" + align1
            align2 = seq2[j - 1] + align2
            j -= 1

    # Finish tracing up or left if any remain
    while i > 0:
        align1 = seq1[i - 1] + align1
        align2 = "-" + align2
        i -= 1
    while j > 0:
        align1 = "-" + align1
        align2 = seq2[j - 1] + align2
        j -= 1

    # Final alignment score is the value at the bottom-right of the score matrix
    alignment_score = score_matrix[m][n]

    return align1, align2, df_score, alignment_score

# Example usage
seq1 = "ATGCATGC"
seq2 = "ATGCATGC"
align1, align2, matrix, score = needleman_wunsch(seq1, seq2)

print("Alignment 1:", align1)
print("Alignment 2:", align2)
print("\nScore Matrix:")
print(matrix)
print("\nAlignment Score:", score)


Alignment 1: ATGCATGC
Alignment 2: ATGCATGC

Score Matrix:
    -   A   T  G  C   A   T   G   C
-   0  -2  -4 -6 -8 -10 -12 -14 -16
A  -2   1  -1 -3 -5  -7  -9 -11 -13
T  -4  -1   2  0 -2  -4  -6  -8 -10
G  -6  -3   0  3  1  -1  -3  -5  -7
C  -8  -5  -2  1  4   2   0  -2  -4
A -10  -7  -4 -1  2   5   3   1  -1
T -12  -9  -6 -3  0   3   6   4   2
G -14 -11  -8 -5 -2   1   4   7   5
C -16 -13 -10 -7 -4  -1   2   5   8

Alignment Score: 8


In [1]:
import numpy as np
import pandas as pd

# scoring setup
Match_score = 1
Mismatch_penalty = -1
Gap_penalty = -2

def needleman_wunsch(seq1, seq2):
    # Initialize scoring matrix
    m = len(seq1)
    n = len(seq2)
    score_matrix = np.zeros((m+1, n+1), dtype=int)
    backtrace_matrix = np.zeros((m+1, n+1), dtype=int)

    # Fill the first row and column with gap penalties
    for i in range(m + 1):
        score_matrix[i][0] = Gap_penalty * i
    for j in range(n + 1):
        score_matrix[0][j] = Gap_penalty * j

    # Fill the scoring matrix and backtrace matrix
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            match = score_matrix[i - 1][j - 1] + (Match_score if seq1[i - 1] == seq2[j - 1] else Mismatch_penalty)
            delete = score_matrix[i - 1][j] + Gap_penalty
            insert = score_matrix[i][j - 1] + Gap_penalty

            # Find maximum value and update the score matrix
            score_matrix[i][j] = max(match, delete, insert)

            # Update the backtrace matrix to record the direction
            if score_matrix[i][j] == match:
                backtrace_matrix[i][j] = 1  # Diagonal (match/mismatch)
            elif score_matrix[i][j] == delete:
                backtrace_matrix[i][j] = 2  # Top
            else:
                backtrace_matrix[i][j] = 3  # Left

    # Convert score matrix to DataFrame for better visualization
    df_score = pd.DataFrame(score_matrix, index=["-"] + list(seq1), columns=["-"] + list(seq2))

    # Traceback to get alignment
    align1, align2 = "", ""
    i, j = m, n

    # Trace back using the backtrace matrix
    while i > 0 and j > 0:
        if backtrace_matrix[i][j] == 1:
            align1 = seq1[i - 1] + align1
            align2 = seq2[j - 1] + align2
            i -= 1
            j -= 1
        elif backtrace_matrix[i][j] == 2:
            align1 = seq1[i - 1] + align1
            align2 = "-" + align2
            i -= 1
        else:
            align1 = "-" + align1
            align2 = seq2[j - 1] + align2
            j -= 1

    # Finish tracing up or left if any remain
    while i > 0:
        align1 = seq1[i - 1] + align1
        align2 = "-" + align2
        i -= 1
    while j > 0:
        align1 = "-" + align1
        align2 = seq2[j - 1] + align2
        j -= 1

    # Alignment score is the value in the bottom-right of the score matrix
    alignment_score = score_matrix[m][n]

    return align1, align2, alignment_score, df_score, backtrace_matrix

# Example usage
seq1 = "GATTACA"
seq2 = "GCATGCU"
align1, align2, alignment_score, matrix, backtrace_matrix = needleman_wunsch(seq1, seq2)

print("Alignment 1:", align1)
print("Alignment 2:", align2)
print("\nAlignment Score:", alignment_score)
print("\nScore Matrix:")
print(matrix)
print("\nBacktrace Matrix:")
print(backtrace_matrix)


Alignment 1: GATTACA
Alignment 2: GCATGCU

Alignment Score: -1

Score Matrix:
    -   G  C  A  T   G   C   U
-   0  -2 -4 -6 -8 -10 -12 -14
G  -2   1 -1 -3 -5  -7  -9 -11
A  -4  -1  0  0 -2  -4  -6  -8
T  -6  -3 -2 -1  1  -1  -3  -5
T  -8  -5 -4 -3  0   0  -2  -4
A -10  -7 -6 -3 -2  -1  -1  -3
C -12  -9 -6 -5 -4  -3   0  -2
A -14 -11 -8 -5 -6  -5  -2  -1

Backtrace Matrix:
[[0 0 0 0 0 0 0 0]
 [0 1 3 3 3 1 3 3]
 [0 2 1 1 3 3 3 3]
 [0 2 1 1 1 3 3 3]
 [0 2 1 1 1 1 1 1]
 [0 2 1 1 2 1 1 1]
 [0 2 1 2 1 1 1 1]
 [0 2 2 1 1 1 2 1]]
